In [9]:
import googlemaps
import pandas as pd
import time

In [10]:
with open('api_key.txt', 'r') as file:
    lines = [line.rstrip() for line in file]
    API_KEY = lines[0]

In [11]:
cities = ["Gdańsk", "Gdynia", 'Sopot']
food_types = ["italian", "turkish", "indian", "chinese", "sushi", "pizza", "mexican", "japanese", "french"]

In [12]:
def get_restaurants(food_type, city, api_key):
    # Initialize the client
    gmaps = googlemaps.Client(key=api_key)
    
    # Perform the initial search
    results = gmaps.places(f"{food_type} restaurants in {city}")
    all_restaurants = results.get('results', [])
    
    restaurant_df = pd.DataFrame(columns=["City", "Place_ID", "Place_Name", "Cuisine"])
    for restaurant in all_restaurants:
        insert_line = [city, restaurant.get('place_id'), restaurant.get('name'), food_type]
        restaurant_df.loc[len(restaurant_df)] = insert_line 
    
    return restaurant_df

In [13]:
all_restaurants_df = pd.DataFrame(columns=["City", "Place_ID", "Place_Name", "Cuisine"])
for city in cities:
    for food in food_types:
        df = get_restaurants(food, city, API_KEY)
        all_restaurants_df = pd.concat([all_restaurants_df, df])

In [14]:
all_restaurants_df.reset_index(drop=True, inplace=True)
all_restaurants_df.drop_duplicates()
all_restaurants_df

,City,Place_ID,Place_Name,Cuisine
0,Gdańsk,ChIJEViZB3Zz_UYRtc7Vpy-JIAA,Ristorante I Pazzi,italian
1,Gdańsk,ChIJy0V-UtF1_UYRAUkwrC67Iwc,Ferrara Cucina Italiana,italian
2,Gdańsk,ChIJiUFLSV11_UYRNSX1hpA-HWY,Al Bacio - Pizza Napoletana Pasta & Vino,italian
3,Gdańsk,ChIJRUQylIJ0_UYRk4ByxR1B8zg,Restauracja Trattoria La Cantina Kuchnia włoska,italian
4,Gdańsk,ChIJF7dc7xxz_UYRoLbLELbsfK8,Magari Gdańsk,italian
...,...,...,...,...
396,Sopot,ChIJf1pjw6QL_UYRK7qDFMiXFfs,Ume Sushi,japanese
397,Sopot,ChIJYfhVCJIK_UYRC_zJVeeS8jY,Petit Paris,french
398,Sopot,ChIJ00hv85YK_UYRjUOGCBBBkqA,Restauracja L’ Entre Villes | wesele Sopot | k...,french
399,Sopot,ChIJWTLz6PEL_UYRHIgRHqufIY8,Restaurant L'idillio,french


In [24]:
merged_cuisine_df = (
    all_restaurants_df.groupby(['City', 'Place_ID', 'Place_Name'], as_index=False)
    .agg({'Cuisine': lambda x: list(set(x))})
)
merged_cuisine_df.reset_index(drop=True, inplace=True)
merged_cuisine_df

,City,Place_ID,Place_Name,Cuisine
0,Gdańsk,ChIJ-3Cf_tJ1_UYRP43jW3p9NBc,Sun Ramen,[chinese]
1,Gdańsk,ChIJ-zxRgCV1_UYRwc1w2WONNOo,KEBAB ARARAT SMAKI ORIENTU,[turkish]
2,Gdańsk,ChIJ07w_7sZ1_UYRJhfK1gDFuW4,Chang - Thai Street Food,[chinese]
3,Gdańsk,ChIJ1RQXZGtz_UYR7s7HS0ofYLQ,Great Wall Chinese Restaurant,[chinese]
4,Gdańsk,ChIJ1TlUK0MB_UYRa0nrSV9GkXg,MEXICAGA,[mexian]
...,...,...,...,...
297,Sopot,ChIJxbprrO51_UYRMXlk2zEEzz4,Kebab DRWAL,[turkish]
298,Sopot,ChIJxwrrxrAL_UYRgTk96K2MUK0,Fiszteria = ryby + pizza,[pizza]
299,Sopot,ChIJz-QrWlcL_UYRVkLgwQKP6hQ,Hona,"[japanese, sushi]"
300,Sopot,ChIJzZf1KZcK_UYRT1jcgb8x9h0,Kuchnia Wietnamska,[chinese]


In [26]:
merged_cuisine_df.sort_values(["Place_Name"]).to_csv("list_of_restaurant.csv", sep=";")

In [25]:
merged_cuisine_df.sort_values(["Place_Name"])

,City,Place_ID,Place_Name,Cuisine
154,Gdynia,ChIJK4PtjPun_UYRRYuECz2jxWk,10/10 Sushi,"[japanese, sushi]"
294,Sopot,ChIJvybH-5IK_UYRfeQa5nqUcfA,1911 Restaurant,"[turkish, chinese]"
269,Sopot,ChIJZTqg4JEK_UYReRYIAngM-9k,77 Sushi Sopot,[sushi]
256,Sopot,ChIJR_4eirUL_UYRCEVs1GUPXdY,A MODO MIO Da CIRO SOPOT,[italian]
109,Gdańsk,ChIJoypSPaFz_UYRe_809t6YJOQ,A la française,[french]
...,...,...,...,...
171,Gdynia,ChIJU0R9dJCn_UYR2RUow9ZYKr0,sushi.fresh,"[japanese, sushi]"
245,Sopot,ChIJ7ZaTw5EK_UYRO3MOsVS1e_0,Śliwka w Kompot,"[italian, turkish, pizza, chinese]"
231,Gdynia,ChIJxZgXuCuh_UYRXrGeVk7yKMc,Świeżo Sushi,"[japanese, sushi]"
11,Gdańsk,ChIJ3wv4m691_UYRMF7XZ3n62wg,„Bardzo Włosko!” Pizza i Wino,[italian]
